In [3]:
from __future__ import annotations
from abc import ABCMeta, abstractmethod
from dataclasses import dataclass, field

import numpy as np
import typing as tp


Tn = tp.TypeVar("Tn", "DataNode", "FunctionNode")
Tc = tp.TypeVar("Tc", list[Tn], None)

@dataclass
class DataNode(tp.Generic[Tn]):
    data: tp.Any = None
    name: str | None = None
    children: list[Tn] = field(default_factory=list)
    
    
class FunctionNode(tp.Generic[Tn]):
    def __init__(
        self, 
        name: str | None = None, 
        children: list[Tn] | None = None
    ) -> None:
        self.name = name
        if children is None:
            self.children = []
        else:
            self.children = children


In [10]:
T = tp.TypeVar("T", str, int, float, list[int])

def f(x: T):
    print(x)
    
f(1)
f([1])
f([1.0])

1
[1]
[1.0]


In [6]:
Tn = tp.TypeVar("Tn", "DataNode", "FunctionNode")
Tc = tp.TypeVar("Tc", list[Tn], None)
Tc = list[Tn]


In [85]:
import uuid
import numpy as np
import pandas as pd


@dataclass
class ID(DataNode[T_Node]):
    data: str = str(uuid.uuid4())

@dataclass
class Array(DataNode[T_Node]):
    data: np.ndarray
    
    def min(self) -> float:
        return float(self.data.min())
    
    def max(self) -> float:
        return float(self.data.max())
    
    def mean(self) -> float:
        return float(self.data.mean())
    
    def std(self) -> float:
        return float(self.data.std())
    
    
@dataclass
class Energy(Array):
    name: str = "energy"

@dataclass
class Objective(Array):
    name: str = "objective"

@dataclass
class ConstraintViolation(Array):
    pass

class Record(FunctionNode[DataNode]):
    def __init__(self, name: str | None = None, children: list[DataNode] | None = None) -> None:
        super().__init__(name, children)
    
    def to_dict(self) -> dict:
        if self.children is None:
            raise TypeError("Value of attribute 'children' is None. Therefore, it cannot be converted to dict.")
        else:
            return {node.name: node.data for node in self.children}
            
    def to_series(self) -> pd.Series:
        return pd.Series(self.to_dict())

class Table(FunctionNode):
    def __init__(self, name: str | None = None, children: list[DataNode | FunctionNode] | None = None) -> None:
        super().__init__(name, children)
    
    
    def to_dataframe(self) -> pd.DataFrame:
        return pd.concat([record.to_series() for record in self.children], axis=1)
    
class Artifact(FunctionNode):
    children: list[Record]
    
    def to_dict(self):
        pass
    


In [48]:
import jijzept as jz
import jijmodeling as jm

sampler = jz.JijSASampler(config="/home/d/.jijzept/config.toml")

problem = jm.Problem("sample")
x = jm.Binary("x", 5)
problem += x[:]
problem += jm.Constraint("onehot", x[:] == 1)

res = sampler.sample_model(problem, {})

In [4]:
res.feasible()

SampleSet(record=Record(solution={'x': [(([1],), [1], (5,))]}, num_occurrences=[1]), evaluation=Evaluation(energy=[0.0], objective=[1.0], constraint_violations={'onehot': [0.0]}, penalty={}), measuring_time=MeasuringTime(solve=SolvingTime(preprocess=None, solve=0.010204792022705078, postprocess=None), system=SystemTime(post_problem_and_instance_data=1.143580675125122, request_queue=0.3961308002471924, fetch_problem_and_instance_data=None, fetch_result=3.2182419300079346, deserialize_solution=8.535385131835938e-05), total=4.765757083892822))

In [81]:
b_id = ID(name="benchmark_id")
energy = Energy(np.array(res.evaluation.energy))
objective = Objective(np.array(res.evaluation.objective))
if res.evaluation.constraint_violations is not None:
    const_name, const_values  = list(res.evaluation.constraint_violations.items())[0]
    constraint_violation = ConstraintViolation(np.array(const_values), const_name)
else:
    constraint_violation = ConstraintViolation()

print(energy)
print(objective)
print(constraint_violation)

Energy(data=array([0.]), name='energy', children=[])
Objective(data=array([1.]), name='objective', children=[])
ConstraintViolation(data=array([0.]), name='onehot', children=[])


In [86]:
record = Record(
    name="0",
    children=[b_id, energy, objective, constraint_violation]
)


In [83]:
record.to_dict()

{'benchmark_id': '73a1a2de-d313-42ff-975f-b6f3843cbafa',
 'energy': array([0.]),
 'objective': array([1.]),
 'onehot': array([0.])}

In [84]:
record.to_series()

benchmark_id    73a1a2de-d313-42ff-975f-b6f3843cbafa
energy                                         [0.0]
objective                                      [1.0]
onehot                                         [0.0]
dtype: object

In [ ]:
table = Table(children=[])

In [67]:
type(float(np.zeros(10).mean().astype(float))) == float

True

In [1]:
type(np.zeros((10, 10)).mean(axis=0)) == np.ndarray

NameError: name 'np' is not defined

In [37]:
np.array([1, 2, 3])

array([1, 2, 3])

In [31]:
import numpy as np
import pandas as pd

df = pd.DataFrame()
df.at[0, "numpy"] = object
df.at[0, "numpy"] = np.random.uniform(0, 10, (5, 5, 5))
df.at[0, "numpy"]
df.to_csv("test.csv")
df

,numpy
0,"[[[5.223747479002759, 4.800946769819401, 6.116..."


In [32]:
df2 = pd.read_csv("test.csv", index_col=0)
df2.at[0, "numpy"]

'[[[5.22374748 4.80094677 6.11689071 7.31566368 1.23736595]\n  [0.28428062 9.44307097 1.89903436 4.19765826 5.59597613]\n  [9.47758591 2.04675904 8.46765006 1.51294084 1.91436422]\n  [9.17592769 9.4342569  7.30417504 9.27726754 6.93338604]\n  [2.37851468 2.273678   0.72553692 8.81048314 0.96157555]]\n\n [[0.47678681 6.60742358 5.60403739 7.47040213 2.25139269]\n  [6.62702913 3.73385153 5.06712167 2.85190424 8.50040324]\n  [6.65611956 0.01661034 6.66244394 3.16240103 5.04807354]\n  [1.96589634 3.25976464 9.91987268 5.14661533 0.54151853]\n  [1.67158872 8.4845318  0.10886078 3.44503153 6.71858885]]\n\n [[8.7607871  4.32735197 2.62176054 1.76345339 3.07191038]\n  [3.61556733 7.33879381 7.02674854 4.08777969 6.84802105]\n  [5.05147354 4.39866683 1.26395654 8.07813761 7.84091721]\n  [5.98800736 3.51749256 3.64328247 1.14554302 9.41778514]\n  [7.81939986 9.79472568 2.47649905 2.23550822 8.37572771]]\n\n [[0.34491114 1.37364409 5.67661407 8.54948312 8.16806013]\n  [1.7780197  0.73956253 4.432

In [33]:
import re

def _numpystr_to_liststr(x):
    try:
        return re.sub(
            r"\n+",
            "",
            re.sub(
                r" +",
                ", ",
                re.sub(
                    r"\[ +",
                    "[",
                    re.sub(
                        r" +\]",
                        "]",
                        re.sub(r"\.\]", ".0]", re.sub(r"\. ", ".0 ", x)),
                    ),
                ),
            ),
        )
    except TypeError:
        return x


In [34]:
import json

obj = json.loads(_numpystr_to_liststr(df2.at[0, "numpy"]))
print(type(obj))
obj = np.array(obj)
obj

<class 'list'>


array([[[5.22374748, 4.80094677, 6.11689071, 7.31566368, 1.23736595],
        [0.28428062, 9.44307097, 1.89903436, 4.19765826, 5.59597613],
        [9.47758591, 2.04675904, 8.46765006, 1.51294084, 1.91436422],
        [9.17592769, 9.4342569 , 7.30417504, 9.27726754, 6.93338604],
        [2.37851468, 2.273678  , 0.72553692, 8.81048314, 0.96157555]],

       [[0.47678681, 6.60742358, 5.60403739, 7.47040213, 2.25139269],
        [6.62702913, 3.73385153, 5.06712167, 2.85190424, 8.50040324],
        [6.65611956, 0.01661034, 6.66244394, 3.16240103, 5.04807354],
        [1.96589634, 3.25976464, 9.91987268, 5.14661533, 0.54151853],
        [1.67158872, 8.4845318 , 0.10886078, 3.44503153, 6.71858885]],

       [[8.7607871 , 4.32735197, 2.62176054, 1.76345339, 3.07191038],
        [3.61556733, 7.33879381, 7.02674854, 4.08777969, 6.84802105],
        [5.05147354, 4.39866683, 1.26395654, 8.07813761, 7.84091721],
        [5.98800736, 3.51749256, 3.64328247, 1.14554302, 9.41778514],
        [7.81939